# Attendance System with Facial Recognition


In [ ]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

In [ ]:
imagePath = 'Images'
Sharpening_Images = 'SharpeningImages'
images = [] #List for all images
classNames = [] #List for names for each image for example: Mohamed Khairi
PersonList = os.listdir(imagePath) #List contains all the images in the Images File, used to get list of all directories in the specidied directory

In [ ]:
for i in PersonList:
    currentPerson = cv2.imread(f'{imagePath}/{i}') #read all images inside Images file
    gaussianBlur = cv2.GaussianBlur(currentPerson, (7, 7), 2) #Blur the Image, (7, 7) means gaussian kernel 7x7 and the higher it is the greater the blur, '2' is standard deviation the higher it is the greater the blur + Control amount of the blurring
    sharpened = cv2.addWeighted(currentPerson, 1.5, gaussianBlur, -0.5, 0) #Sharpening the Image, (src1, alpha, src2, beta, gamma), el alpha - beta = 1 (should be = 1)
    cv2.imwrite(f'{Sharpening_Images}/{i}', sharpened) #Save the sharpened image
    images.append(sharpened) #append in images list
    classNames.append(os.path.splitext(i)[0]) #Take the name from imaage and append in classNames list, split before point . and take what comes before it [0], for not taking 'jpg'
print(classNames)

In [ ]:
def findEncoding(image):
    encodeList = []
    for i in images: #to read all images in list
        i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB) #Convert image from BGR to RGB, because cv2 read BGR, and Face Recognition the image should be RGB  
        encode = face_recognition.face_encodings(i)[0] # 'Distinctive points' to recognize when see them, It starts by placing many points on the face such as the mouth and the eyes
        encodeList.append(encode) #Append to the encodeList List contains encode for each image
    return encodeList

In [ ]:
def markAttendance(name):
    with open("Attendance.csv", "r+") as f: #r+, Read and Write at the same time
        myDataList = f.readlines() #Read each line in the file and put it in the myDataList
        nameList = [] #To put names of the persons that in attendance sheet to avoid duplicates in sheet
        for line in myDataList:
            entry = line.split(",")
            nameList.append(entry[0]) # [0] means the name, after split the line
        if name not in nameList: #To avoid duplicates
            now = datetime.now()
            Clock = now.strftime("%H:%M:%S") #String Format Time : To convert the 'now' datetime into string [ Clock ]
            Date = now.strftime("%Y-%m-%d") #String Format Time : To convert the 'now' datetime into string [ Date ]
            f.writelines(f'\n{name}, {Date}, {Clock}') #insert new record in the attendance sheet

In [ ]:
encodeListKnown = findEncoding(images) #All the details and patterns of the faces are put in encodeListKnown, this list contains the known faces
print("- Encoding Complete ...")

In [ ]:
capture = cv2.VideoCapture(0) #0: laptop camera, 1: external camera

while True:
    _, img = capture.read() #Read all frames in the form of image
    
    imgResize = cv2.resize(img, None, fx=0.25, fy=0.25, interpolation=cv2.INTER_AREA) #To reduced image by 25% to make the video faster, Beacause it will take a lot of time to encode the original image
    imgResize = cv2.cvtColor(imgResize, cv2.COLOR_BGR2RGB) #BGR TO RGB
    
    faceCurrentFrame = face_recognition.face_locations(imgResize) #Location of the face from the video and make bounding boxes of human faces in a video
    encodeCurrentFrame = face_recognition.face_encodings(imgResize, faceCurrentFrame) #Encode face in the new smaller image(25%) with the its location
    
    for encodeFace, faceLocation in zip(encodeCurrentFrame, faceCurrentFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace) #Compare the new encode(face) with encodeListKnown(this list contains the known faces)

        #Given a list of face encodings, compare them to a known face encoding and get a euclidean distance for each comparison face. 
        #The distance tells you how similar the faces are.
        faceDistance = face_recognition.face_distance(encodeListKnown, encodeFace) #List contains 5 things(faces), for example 2nd image Elon Mask and put camera on him the 2nd element will be samller one so this is Elon Mask
        matchIndex = np.argmin(faceDistance) #Take smallest element in faceDistance List to know who is in the video + holds the index of the known face with the lowest distance score (the best match to the detected face)
        
        confidence = 100 - faceDistance[matchIndex] * 100 #to calculate the confidence as percentage, whenever the value for face Distance lower, the confidence higher and be 0.1, 0.2, ... 1
        if matches[matchIndex]: #If this face is known and in the known list
            name = classNames[matchIndex].upper()
            #y1: Represents the top coordinate (y-axis) of the bounding box
            #x2: Represents the right coordinate (x-axis) of the bounding box
            #y2: Represents the bottom coordinate (y-axis) of the bounding box
            #x1: Represents the left coordinate (x-axis) of the bounding box
            y1, x2, y2, x1 = faceLocation
            y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4 #Because we want to display the big picture so we multiply 25 * 4, to return to the original image
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2) #Drawing a rectangle on each known face
            
            cv2.putText(img, f"{confidence: .2f}%", (x1 + 6, y1 - 6), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2) #To display the confidence value on the rectangle

            if confidence >= 50: #50%
                print(name)
                #Drawing a small rectangle below the big rectangle to put on it the name of the person/face 
                cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 0, 255), cv2.FILLED)
                cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                markAttendance(name) #Pass the name of the person/face who was recogonized to be recorded in attendance sheet
            else:
                #Drawing a small rectangle below the big rectangle to put on it " Get Closer !! "
                cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 0, 255), cv2.FILLED)
                cv2.putText(img, "Get Closer !!", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
    
    
    
    cv2.imshow("Face Recognition", img)
    cv2.waitKey(1) #0: Image, 1: Video
